In [25]:
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from PIL import Image

In [2]:
tf.__version__

'2.0.0'

In [3]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('X_train.shape:',X_train.shape)
print('X_test.shape:',X_test.shape)
print('y_train.shape:',y_train.shape)
print('y_test.shape:',y_test.shape)

X_train.shape: (60000, 28, 28)
X_test.shape: (10000, 28, 28)
y_train.shape: (60000,)
y_test.shape: (10000,)


In [8]:
n_classes = 10
def onehot(y,start,end,categories='auto'):
    ohot = OneHotEncoder()
    a = np.linspace(start,end-1,end-start)
    b = np.reshape(a,[-1,1]).astype(np.int32)
    ohot.fit(b)
    c = ohot.transform(y).toarray()
    return c
def MNISTLable_TO_ONEHOT(X_Train,Y_Train,X_Test,Y_Test,shuff=True):
    Y_Train = np.reshape(Y_Train,[-1,1])
    Y_Test = np.reshape(Y_Test,[-1,1])
    Y_Train = onehot(Y_Train.astype(np.int32),0,n_classes)
    Y_Test = onehot(Y_Test.astype(np.int32),0,n_classes)
    if shuff ==True:
        X_Train,Y_Train = shuffle(X_Train,Y_Train)
        X_Test,Y_Test = shuffle(X_Test,Y_Test)
        return X_Train,Y_Train,X_Test,Y_Test

In [9]:
X_train,y_train,X_test,y_test = MNISTLable_TO_ONEHOT(X_train,y_train,X_test,y_test)

/Users/mingzema/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/mingzema/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "cate

In [21]:
batch_size = 16
i = 0
batch_x = X_train[i*batch_size:(i+1)*batch_size,:]
batch_x = np.reshape(batch_x,[-1,28*28])
batch_y = y_train[i*batch_size:(i+1)*batch_size,:]

In [22]:
for j in batch_x:
    print(j)
    print()

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0  42 233 219 125 125
 125 125  42   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  46 234 254 255 254 254 254 251 149   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 119 143
 143 143 143 143 218 222   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 170 239  51   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  18 203 222   0   0   0   0   

In [26]:
batch_size = 16
def rot_generator(batch_size, frames, frames_input=3):
#     batch1in, batch1_labels = mnist.train.next_batch(batch_size)
    i = 0
    batch1in = X_train[i*batch_size:(i+1)*batch_size,:]
    batch1in = np.reshape(batch_x,[-1,28*28])
    batch1_labels = y_train[i*batch_size:(i+1)*batch_size,:]
    
    batch1 = batch1in.reshape([batch_size, 28, 28])
    batch2 = batch1in.reshape([batch_size, 28, 28])
    batch = np.zeros([batch_size, 64, 64, 3 * frames_input])
    batch_gen = np.zeros([batch_size, 64, 64, 3 * frames])
    batch_old = np.zeros([batch_size, 64, 64, 3 * frames])
    batch_labels = np.zeros([batch_size, 13])
    batch_labels[:, :10] += batch1_labels
    text_labels = np.zeros([batch_size, 5])
    for i in range(batch_size):
        t = np.random.randint(0, 32 // (frames + 2) + 1)
        l = np.random.randint(0, 256, [3]).astype(float) / 255
        batch_labels[i, 10:] = l
        random = np.random.randint(0, 4)
        rot = np.random.normal(0, 5)
        if t == 0:
            text_labels[i] = np.array([rot, -1, 1, 1, -1])
            text_labels[i][-1] *= random
            text_labels[i][-2] *= random
            for r in range(frames_input):
                batch2[i] = (np.array(
                    Image.fromarray(batch1[i] * 255.).rotate(r * rot, Image.BILINEAR).getdata()) / 255.).reshape(28, 28)
                for j in range(3):
                    batch[i, 2 + (random * r):30 + (random * r), 2 + (random * r):30 + (random * r), j + (3 * r)] = \
                        batch2[i] * l[j]
                    if r == (frames_input - 1):
                        batch_old[i, 2 + (random * r):30 + (random * r), 2 + (random * r):30 + (random * r), j] = \
                            batch2[i] * l[j]
            for r in range(frames):
                batch2[i] = (np.array(
                    Image.fromarray(batch1[i] * 255.).rotate((j + 3) * rot, Image.BILINEAR).getdata()) / 255.).reshape(
                    28, 28)
                for j in range(3):
                    batch_gen[i, 2 + (random * frames_input) + (random * r):30 + (frames_input * random) + (random * r),
                    2 + (random * frames_input) + (random * r):30 + (frames_input * random) + (random * r),
                    j + (3 * r)] = batch2[i] * l[j]
                    if r != (frames - 1):
                        batch_old[i, 2 + (random * r):30 + (random * r), 2 + (random * r):30 + (random * r),
                        j + (3 * r) + 3] = batch2[i] * l[j]
        elif t == 1:
            text_labels[i] = np.array([rot, 1, -1, -1, 1])
            text_labels[i][-1] *= random
            text_labels[i][-2] *= random
            for r in range(3):
                batch2[i] = (np.array(
                    Image.fromarray(batch1[i] * 255.).rotate(r * rot, Image.BILINEAR).getdata()) / 255.).reshape(28, 28)
                for j in range(3):
                    batch[i, 34 - (random * r):62 - (random * r), 34 - (random * r):62 - (random * r), j + (3 * r)] = \
                        batch2[i] * l[j]
                    if r == (frames_input - 1):
                        batch_old[i, 2 + (random * r):30 + (random * r), 2 + (random * r):30 + (random * r), j] = \
                            batch2[i] * l[j]
            for r in range(frames):
                batch2[i] = (np.array(
                    Image.fromarray(batch1[i] * 255.).rotate((j + 3) * rot, Image.BILINEAR).getdata()) / 255.).reshape(
                    28, 28)
                for j in range(3):
                    batch_gen[i,
                    34 - (frames_input * random) - (random * r):62 - (random * frames_input) - (random * r),
                    34 - (frames_input * random) - (random * r):62 - (random * frames_input) - (random * r),
                    j + (3 * r)] = batch2[i] * l[j]
                    if r != (frames - 1):
                        batch_old[i, 2 + (random * r):30 + (random * r), 2 + (random * r):30 + (random * r),
                        j + (3 * r) + 3] = batch2[i] * l[j]
        elif t == 2:
            text_labels[i] = np.array([rot, -1, -1, 1, 1])
            text_labels[i][-1] *= random
            text_labels[i][-2] *= random
            for r in range(3):
                batch2[i] = (np.array(
                    Image.fromarray(batch1[i] * 255.).rotate(r * rot, Image.BILINEAR).getdata()) / 255.).reshape(28, 28)
                for j in range(3):
                    batch[i, 34 - (random * r):62 - (random * r), 2 + (random * r):30 + (random * r), j + (3 * r)] = \
                        batch2[i] * l[j]
                    if r == (frames_input - 1):
                        batch_old[i, 2 + (random * r):30 + (random * r), 2 + (random * r):30 + (random * r), j] = \
                            batch2[i] * l[j]
            for r in range(frames):
                batch2[i] = (np.array(
                    Image.fromarray(batch1[i] * 255.).rotate((j + 3) * rot, Image.BILINEAR).getdata()) / 255.).reshape(
                    28, 28)
                for j in range(3):
                    batch_gen[i,
                    34 - (frames_input * random) - (random * r):62 - (random * frames_input) - (random * r),
                    2 + (random * frames_input) + (random * r):30 + (frames_input * random) + (random * r),
                    j + (3 * r)] = batch2[i] * l[j]
                    if r != (frames - 1):
                        batch_old[i, 2 + (random * r):30 + (random * r), 2 + (random * r):30 + (random * r),
                        j + (3 * r) + 3] = batch2[i] * l[j]
        else:
            text_labels[i] = np.array([rot, 1, 1, -1, -1])
            text_labels[i][-1] *= random
            text_labels[i][-2] *= random
            for r in range(3):
                batch2[i] = (np.array(
                    Image.fromarray(batch1[i] * 255.).rotate(r * rot, Image.BILINEAR).getdata()) / 255.).reshape(28, 28)
                for j in range(3):
                    batch[i, 2 + (random * r):30 + (random * r), 34 - (random * r):62 - (random * r), j + (3 * r)] = \
                        batch2[i] * l[j]
                    if r == (frames_input - 1):
                        batch_old[i, 2 + (random * r):30 + (random * r), 2 + (random * r):30 + (random * r), j] = \
                            batch2[i] * l[j]
            for r in range(frames):
                batch2[i] = (np.array(
                    Image.fromarray(batch1[i] * 255.).rotate((j + 3) * rot, Image.BILINEAR).getdata()) / 255.).reshape(
                    28, 28)
                for j in range(3):
                    batch_gen[i, 2 + (random * frames_input) + (random * r):30 + (frames_input * random) + (random * r),
                    34 - (frames_input * random) - (random * r):62 - (random * frames_input) - (random * r),
                    j + (3 * r)] = batch2[i] * l[j]
                    if r != (frames - 1):
                        batch_old[i, 2 + (random * r):30 + (random * r), 2 + (random * r):30 + (random * r),
                        j + (3 * r) + 3] = batch2[i] * l[j]
    return batch, batch_old, batch_gen, batch_labels, text_labels


In [27]:
frames=5
frames_input = 3
image_sample, image_old,image_gen,image_labels, text_labels = rot_generator(batch_size, frames, frames_input=3)

In [28]:
image_sample

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    